# Time resample

In [1]:
import pandas as pd
from read_json import as_json, format_time
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("data_samples/data_sample_4.csv")
data["@timestamp"] = pd.to_datetime(data["@timestamp"].map(format_time))  # convert to pandas datetime
id_nan =data.system.isna()
data_no_nan = data.loc[~id_nan]

In [3]:
hostnames = data["host"].unique()            # get unique names
hostnames = list(map(as_json, hostnames))
hostnames

[{'name': 'PRUEBAS'},
 {'name': 'PREPRODUCCION'},
 {'name': 'DEVELOP'},
 {'name': 'QASERVER'},
 {'name': 'PRODUCCION'}]

In [5]:
cpu_df

,cores,user_pct,idle_pct,system_pct,host
2,8,0.009163,0.966275,0.024575,PRODUCCION
3,4,0.003125,0.989475,0.007425,DEVELOP
6,4,0.035175,0.929675,0.035150,QASERVER
16,4,0.024650,0.954225,0.021125,PREPRODUCCION
18,8,0.001175,0.994925,0.003900,PRODUCCION
...,...,...,...,...,...
99975,8,0.013450,0.961588,0.024962,PRODUCCION
99984,8,0.008613,0.975150,0.016237,PRODUCCION
99988,8,0.000975,0.995513,0.003512,PRODUCCION
99989,16,0.001125,0.997687,0.001187,PRUEBAS


In [4]:
cpu_df = pd.read_csv("cpu_df.csv",index_col=0)
mem_df = pd.read_csv("memory_df.csv", index_col=0)
net_df = pd.read_csv("network_df.csv", index_col=0)

frames = [cpu_df, mem_df, net_df]
df = pd.concat(frames)  # si jala
df.sort_index(inplace=True)
#df["@timestamp"] = data_no_nan["@timestamp"]
df.insert(0, "@timestamp", data_no_nan["@timestamp"])
df_time = df

## DEVELOP
df_develop = df_time.loc[df_time["host"] == "DEVELOP"].drop("host", axis=1)
df_develop_resampled = df_develop.resample('10Min', on="@timestamp").mean()
df_develop_resampled["cores"] = 4
#df_develop_resampled.to_csv("resampled/develop_resampled.csv")

## QASERVER
# do it by host -->  then relate by host
df_qaserver = df_time.loc[df_time["host"] == "QASERVER"].drop("host", axis=1)
# df_time["@timestamp"].resample("5Min")
df_qaserver_resampled = df_qaserver.resample('10Min', on="@timestamp").mean()
df_qaserver_resampled["cores"] = 4  # the server has the same number of cores
#df_qaserver_resampled.to_csv("resampled/qaserver_resampled.csv")


## PRODUCCION
df_produccion = df_time.loc[df_time["host"] == "PRODUCCION"].drop("host", axis=1)
df_produccion_resampled = df_produccion.resample('10Min', on="@timestamp").mean()
df_produccion_resampled["cores"] = 8
#df_produccion_resampled.to_csv("resampled/produccion_resampled.csv")
df_produccion_resampled

## PREPRODUCCION
df_preproduccion = df_time.loc[df_time["host"] == "PREPRODUCCION"].drop("host", axis=1)
df_preproduccion_resampled = df_preproduccion.resample('10Min', on="@timestamp").mean()
df_preproduccion_resampled["cores"] = 4
#df_preproduccion_resampled.to_csv("resampled/preproduccion_resampled.csv")
df_preproduccion_resampled

## PRUEBAS
df_pruebas = df_time.loc[df_time["host"] == "PRUEBAS"].drop("host", axis=1)
df_pruebas_resampled = df_pruebas.resample('10Min', on="@timestamp").mean()
df_pruebas_resampled["cores"] = 16
#df_pruebas_resampled.to_csv("resampled/pruebas_resampled.csv")

data_full = pd.concat([df_develop_resampled,
                       df_produccion_resampled,
                       df_preproduccion_resampled,
                       df_pruebas_resampled,
                       df_qaserver_resampled])
data_full.sort_index(inplace=True)
data_full.to_csv("resampled/data_4_resampled.csv")